#### Imports 

In [4]:
from sagemaker import get_execution_role
from time import gmtime, strftime
import pandas as pd
import sagemaker
import boto3
import time

#### Essentials

In [5]:
bucket = sagemaker.Session().default_bucket()
prefix = 'loan-default-prediction'
region = 'us-east-1'

In [6]:
batch_input = f's3://{bucket}/{prefix}/batch_input/'
batch_input

's3://sagemaker-us-east-1-892313895307/loan-default-prediction/batch_input/'

In [7]:
batch_output = f's3://{bucket}/{prefix}/batch_output/'
batch_output

's3://sagemaker-us-east-1-892313895307/loan-default-prediction/batch_output/'

In [8]:
current_timestamp = strftime('%Y-%m-%d-%H-%M-%S', gmtime())

In [22]:
automl_job_name = 'LOAN-DEFAULT-PREDICTION-3'  # Copy this from the console
model_name = f'autopilot-best-model-{current_timestamp}'
transform_job_name = f'autopilot-batch-job-{current_timestamp}'

In [10]:
session = boto3.Session()
sagemaker_execution_role = get_execution_role()
sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client('sagemaker', region_name=region)
s3_client = boto3.client('s3')

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210527T121473 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


#### Copy batch input data from local to S3

In [11]:
!aws s3 cp ./data/train/loans_unlabeled.csv {batch_input}

upload: data/train/loans_unlabeled.csv to s3://sagemaker-us-east-1-892313895307/loan-default-prediction/batch_input/loans_unlabeled.csv


#### Get the best model using Autopilot job name

In [18]:
best_candidate = sagemaker_client.describe_auto_ml_job(AutoMLJobName=automl_job_name)["BestCandidate"]
best_candidate_name = best_candidate["CandidateName"]
print(f"CandidateName: {best_candidate_name}")
print(f'FinalAutoMLJobObjectiveMetricName: {best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"]}')
print(f'FinalAutoMLJobObjectiveMetricValue: {best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"]}')

CandidateName: tuning-job-1-fe12f12c5a7642a59c-199-88a01bbf
FinalAutoMLJobObjectiveMetricName: validation:f1
FinalAutoMLJobObjectiveMetricValue: 0.6585800051689148


In [21]:
model = sagemaker_client.create_model(Containers=best_candidate["InferenceContainers"], 
                                      ModelName=model_name, 
                                      ExecutionRoleArn=sagemaker_execution_role)

print(f'Model ARN corresponding to the best candidate is : {model["ModelArn"]}')

Model ARN corresponding to the best candidate is : arn:aws:sagemaker:us-east-1:892313895307:model/autopilot-best-model-2021-06-28-19-25-03


In [26]:
transform_input = {
    "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": batch_input}},
    "ContentType": "text/csv",
    "CompressionType": "None",
    "SplitType": "Line",
}

transform_output = {
    "S3OutputPath": batch_output,
}

transform_resources = {"InstanceType": "ml.m5.4xlarge", "InstanceCount": 1}

sagemaker_client.create_transform_job(
    TransformJobName=transform_job_name,
    ModelName=model_name,
    TransformInput=transform_input,
    TransformOutput=transform_output,
    TransformResources=transform_resources,
)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:transform-job/autopilot-batch-job-2021-06-28-19-25-03',
 'ResponseMetadata': {'RequestId': '69a0081d-8878-4d56-8a47-e0aa0a698825',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '69a0081d-8878-4d56-8a47-e0aa0a698825',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Mon, 28 Jun 2021 19:41:11 GMT'},
  'RetryAttempts': 0}}